In [ ]:
#---------------------------------------------------------------------------------------------LIBRARIES--------------------------------------------------------------------------------------------                                                                                         #import OpenCV2 library for image processing and algorithms
import math
import csv 
import numpy as np                                                                                     #import numpy mathematical library
import pandas as pd
import pickle
import random
import matplotlib.pyplot as plt      #import matplotlib library for plotting
from micromlgen import port

from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))                                  #change width of Jupyer Notebook to use the whole window resolution availab

# import the classifier
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from sklearn.datasets import make_classification

import matplotlib.pyplot as plt
from collections import Counter
import plotly.express as px

In [ ]:
def save_statistics(clf, X_test, y_test, csv_filename, confusion_matrix_filename, metrics_filename):
    y_pred = []
    with open(csv_filename, "a", newline='') as fp:
        for row in X_test:
            result = int((str(clf.predict(row.reshape(1, -1))).replace('[','').replace(']','')))
            y_pred.append(result)
            wr = csv.writer(fp, dialect='excel')
            wr.writerow([result])
    fp.close()
    
    #https://www.baeldung.com/cs/multi-class-f1-score
    #https://stackoverflow.com/questions/31421413/how-to-compute-precision-recall-accuracy-and-f1-score-for-the-multiclass-case
    precision, recall, fscore, support = score(y_test.ravel(), y_pred)
    
    #https://towardsdatascience.com/understanding-the-confusion-matrix-from-scikit-learn-c51d88929c79
    #https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9
    metrics = {"Accuracy: " : clf.score(X_test, y_test.ravel()),
               "F1 Score Weighted: " : f1_score(y_test, y_pred, average='weighted'), 
               "F1 Score per Class: " : fscore,
               "Precision per Class: " : precision,
               "Recall per Class: " : recall,
               "Support:  " : support}
    
    cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
    disp.plot()
    plt.savefig(confusion_matrix_filename, dpi=500)                                                                                     #https://stackoverflow.com/questions/9622163/save-plot-to-image-file-instead-of-displaying-it-using-matplotlib
    plt.show()
    
    print("CLASSES: Clear = 0, Rain = 1, Partially Cloudy = 2, Overcast = 3, Snow = 4" + "\n")
    with open(metrics_filename, 'w') as f:
        for k, v in metrics.items():
            f.write(str(k) + str(v) + '\n\n')
            print(str(k) + str(v))
    f.close()
    
def interractive_plot(X, y, a,b,c, interractive):
    classes = {'#ef8a62': '0', '#1cfc03': '1', '#67a9cf': '2', '#fc0303' : '3', '#00ffe5' : '4'}
    legend = {'#ef8a62': 'Fair', '#1cfc03': 'Rain', '#67a9cf': 'Cloudy', '#fc0303' : 'Overcast', '#00ffe5' : 'Snow'}
    colors = dict()
    for j, item in enumerate(y):
        for k, v in classes.items():
            if item == int(v): colors[j] = k
    
    if interractive is True:
        #https://plotly.com/python/3d-scatter-plots/
        fig = px.scatter_3d(X, a, b, c, color=colors, width=1300, height=1000,
                           labels={"x" : "Temperature", "y" : "Humidity", "z" : "Pressure"}, title="CLASSES CLUSTERS")  
        fig.for_each_trace(lambda t: t.update(name = legend[t.name]))
        fig.update_layout(legend_title_text='WEATHER')
        fig.show()
        
    else:
        #https://www.analyticsvidhya.com/blog/2021/10/interactive-plots-in-python-with-plotly-a-complete-guide/  <= great matplotlib article
        fig = plt.figure(figsize = (10, 10))
        ax = plt.axes(projection ="3d")
        kwarg_params = {'linewidth': 1, 'edgecolor': 'black'}
        ax.scatter3D(a, b, c, c=colors, **kwarg_params)
        ax.set_xlabel('Temperature', fontweight ='bold')
        ax.set_ylabel('Humidity', fontweight ='bold')
        ax.set_zlabel('Pressure', fontweight ='bold')
        plt.suptitle("Dataset Resampled with SMOTE")
        plt.show()
        
def oversampling(X_train, y_train, method):
    if method == 1: X_train, y_train = SMOTE().fit_sample(X_train, y_train)
    else: X_train, y_train = ADASYN().fit_sample(X_train, y_train)
    print(sorted(Counter(y_train).items()))
    print("Total training data: ", X_train.size)
    
    return X_train, y_train

In [ ]:
def main():
    #dataset selection and loading
    train_set_name = "weather_data_2000_2019"
    train_dataset = pd.read_csv(("./datasets/" + train_set_name + ".csv"),header=None)
    test_set_name = "weather_data_2020_2021"
    test_dataset = pd.read_csv(("./datasets/" + test_set_name + ".csv"),header=None) 

    print("Your Train Dataset is: ", train_set_name)                                                                       #display dataset name to user
    print("Your Test Dataset is: ", test_set_name)                                                                       #display dataset name to user

    features_selected = 5
    #training features
    temperature_train = train_dataset.iloc[:, 4:5]
    feels_like_train = train_dataset.iloc[:, 7:8]
    dew_point_train = train_dataset.iloc[:, 8:9]
    humidity_train = train_dataset.iloc[:, 9:10]
    pressure_train = train_dataset.iloc[:, 19:20] 
    #uv_index_train = train_dataset.iloc[:, 24:25] 

    #testing data
    temperature_test = test_dataset.iloc[:, 4:5]
    feels_like_test = test_dataset.iloc[:, 7:8]
    dew_point_test = test_dataset.iloc[:, 8:9]
    humidity_true = test_dataset.iloc[:, 9:10]
    pressure_test = test_dataset.iloc[:, 19:20]
    #uv_index_test = test_dataset.iloc[:, 24:25] 

    X_train = pd.concat([temperature_train, feels_like_train, dew_point_train, humidity_train, pressure_train], axis=1)   #, uv_index_train], axis=1)
    X_train.replace(([np.inf, -np.inf], np.nan), inplace=True)                                                            #replace any infinite values with nan
    X_train = X_train.to_numpy()  

    y_train = train_dataset.iloc[:, 30:31].to_numpy()
    y_true = test_dataset.iloc[:, 30:31].to_numpy()

    #https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
    test_data = pd.concat([temperature_test, feels_like_test, dew_point_test, humidity_true, pressure_test], axis=1)       #, uv_index_test], axis=1)       
    test_data.replace(([np.inf, -np.inf], np.nan), inplace=True)                                                           #replace any infinite values with nan
    X_test = test_data.to_numpy()

    #change all nan values in all datasets with the most frequent value of the dataset
    imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imp.fit(X_train)
    imp.fit(y_train)
    imp.fit(X_test)
    imp.fit(y_true)
    
    disp = int(input("Please choose 1 to display the dataset or any button to cotinue without displaying!"))
    if disp == 1:
        display(X_train)
        display(y_train.ravel())

        display(X_test)
    else: print("Not displaying dataset!")   
        
    #----------------------------------------------------------------------------------------
    rebalance = int(input("PLEASE CHOOSE 1 TO REBALANCE DATASET WITH SMOTE. CHOOSE 2 TO REBALANCE DATASET WITH ADASYN" + 
                          "\nPRESS ANYTHING ELSE TO CONTINUE WITH ORIGINAL!\n"))
    if rebalance == 1:
        X_train, y_train = oversampling(X_train, y_train, 1)
    elif rebalance == 2:
        X_train, y_train = oversampling(X_train, y_train, 2)
    else: print("Total training data: ", X_train.size)
    
    interractive_plot(X_train, y_train, X_train[:, 0], X_train[:, 3], X_train[:, 4], True)
    #------------------------------------------------------------------------------------------
    
    choice = int(input("Please choose 1 for Training and 2 for Random CV Search." +
                       "\nPressing 3 will load the optimised Gaussian Naive Bayes Classifier model!\n"))
    if rebalance == 1: clf_type = "default_rebalanced_SMOTE" if choice == 1 else "optimised_rebalanced_SMOTE" 
    elif rebalance == 2: clf_type = "default_rebalanced_ADASYN" if choice == 1 else "optimised_rebalanced_ADASYN" 
    else: clf_type = "default" if choice == 1 else "optimised"
    model_name = "./trained_models/xgb_" + clf_type + "_" + train_set_name + "_" + test_set_name + "_" + str(features_selected) + "f.p"
    confusion_matrix_filename = "./confusion_matrix/xgb_" + str(clf_type) + "_" + train_set_name + "_"  + test_set_name + "_" + str(features_selected) + "f.png"
    metrics_filename = "./metrics/xgb_" + str(clf_type) + "_" + train_set_name + "_"  + test_set_name + "_" + str(features_selected) + "f.txt"
    csv_filename = "./outputs/output_xgb_"  + str(clf_type) + "_" + train_set_name + "_"  + test_set_name + "_" + str(features_selected) + "f.csv"

    if choice == 1:
        # Create XGBoost classifer object and train it on the selected dataset
        xgb = (XGBClassifier(objective = 'multi:softmax', eval_metric = 'merror')).fit(X_train, y_train.ravel())
        save_statistics(xgb, X_test, y_true, csv_filename, confusion_matrix_filename, metrics_filename)

    elif choice == 2:
        #random search: https://www.kaggle.com/tilii7/hyperparameter-grid-search-with-xgboost
        #grid search: https://www.mikulskibartosz.name/xgboost-hyperparameter-tuning-in-python-using-grid-search/
        #label encoder warning: https://stackoverflow.com/questions/45251126/deprecation-warning-on-xgboost-sklearn
        #https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook
        estimator = XGBClassifier(objective = 'multi:softmax', nthread = 20, seed = 42, num_class = 5, eval_metric = 'merror')
        parameters = {'max_depth': range(2, 10, 1),'n_estimators': range(60, 220, 40), 'learning_rate': [0.1, 0.01, 0.05]}

        grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, cv = 5, verbose = True,  refit = True)
        grid_search.fit(X_train, y_train.ravel())
        
        print(grid_search.best_params_)
        print(grid_search.best_estimator_)
        pickle.dump(grid_search.best_estimator_, open(model_name, "wb"))

    elif choice == 3:
        xgb = pickle.load(open(model_name, "rb"))
        print("Parameters Classifier Loaded: ", xgb.get_params())

        save_statistics(xgb, X_test, y_true, csv_filename, confusion_matrix_filename, metrics_filename)

if __name__ == "__main__":                                                                                               #guard boilerplate
    main()